In [1]:
# This notebook will be used to run the model in a way that makes it easier to develop new components of the model
import nasem_dairy as nd
import pandas as pd

def read_csv_input(path_to_file):
    
    animal_input = {}
    equation_selection = {}
    diet_info_data = {'Feedstuff': [], '%_DM_user': []}

    input_data = pd.read_csv(path_to_file)

    for index, row in input_data.iterrows():
        location = row['Location']
        variable = row['Variable']
        value = row['Value']
        
        if location == 'equation_selection':
            equation_selection[variable] = float(value) if value.replace('.', '', 1).isdigit() else value
        elif location == 'animal_input':
            animal_input[variable] = float(value) if value.replace('.', '', 1).isdigit() else value
        elif location == 'diet_info':
            diet_info_data['Feedstuff'].append(variable)
            diet_info_data['%_DM_user'].append(value)
      
    diet_info = pd.DataFrame(diet_info_data)
    diet_info['%_DM_user'] = pd.to_numeric(diet_info['%_DM_user'], downcast="float")

    return diet_info, animal_input, equation_selection


In [2]:

def run_dev_model():
    # This will be calculated by a function I need to write, placeholder for now
    An_DigTPaIn = 1
    An_GasEOut = 1

########################################
# Step 1: Read User Input
########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    diet_info, animal_input, equation_selection = read_csv_input('./input.csv')
    
    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = diet_info['Feedstuff'].tolist()
    feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')

    # Scale %_DM_intake to 100%
    user_perc = diet_info['%_DM_user'].sum()
    scaling_factor = 100 / user_perc

    # Should be called 'Fd_DMInp' instead of %_DM_intake
    diet_info['%_DM_intake'] = diet_info['%_DM_user'] * scaling_factor
    # Adjust so sum is exactly 100
    adjustment = 100 - diet_info['%_DM_intake'].sum()
    diet_info['%_DM_intake'] += adjustment / len(diet_info)

    # Predict DMI
    if equation_selection['DMI_pred'] == 0:
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(animal_input['An_Parity_rl'], animal_input['Trg_MilkProd'], animal_input['An_BW'], animal_input['An_BCS'],
                                                         animal_input['An_LactDay'], animal_input['Trg_MilkFatp'], animal_input['Trg_MilkTPp'], animal_input['Trg_MilkLacp'])
    # Should be called 'Fd_DMIn' for consistency with R code
    diet_info['kg_intake'] = diet_info['%_DM_intake'] / 100 * animal_input['DMI']

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7


########################################
# Step 2: Feed Based Calculations
########################################
    diet_info = nd.get_nutrient_intakes(diet_info, feed_data, animal_input, equation_selection, nd.coeff_dict)

########################################
# Step 3: Microbial Protein Calculations
########################################
    Du_MiN_NRC2021_g = nd.calculate_Du_MiN_g(diet_info.loc['Diet', 'Fd_NDFIn'], animal_input['DMI'], diet_info.loc['Diet', 'Fd_St_kg/d'], diet_info.loc['Diet', 'Fd_CP_kg/d'], diet_info.loc['Diet', 'Fd_ADF_kg/d'], 
                                    diet_info.loc['Diet', 'Fd_ForWetIn'], diet_info.loc['Diet', 'Fd_RUPIn'], diet_info.loc['Diet', 'Fd_ForNDFIn'], diet_info.loc['Diet', 'Dt_RDPIn'], nd.coeff_dict)    
    
########################################
# Step 4: Amino Acid Calculations
########################################
    AA_values, Du_MiCP_g = nd.AA_calculations(Du_MiN_NRC2021_g, feed_data, diet_info, animal_input, nd.coeff_dict)

########################################
# Step 5: Other Calculations
########################################
# Intake calculations that require additional steps, need results from other calculations or values that need to be calculated for other functions

    GrUter_BWgain = nd.calculate_GrUter_BWgain(animal_input['Fet_BWbrth'], animal_input['An_AgeDay'], animal_input['An_GestDay'], animal_input['An_GestLength'], animal_input['An_LactDay'], animal_input['An_Parity_rl'], nd.coeff_dict)

    # This function could be renamed as it is doing all the DE intake calculations
    An_DEIn, An_DENPNCPIn, An_DETPIn, An_DigNDFIn, An_DEStIn, An_DEFAIn, An_DErOMIn, An_DENDFIn, Fe_CP, Fe_CPend_g, Du_idMiCP_g = nd.calculate_An_DEIn(diet_info.loc['Diet', 'Fd_DigNDFIn_Base'], diet_info.loc['Diet', 'Fd_NDFIn'], 
                                                                                                                    diet_info.loc['Diet', 'Fd_DigStIn_Base'], diet_info.loc['Diet', 'Fd_St_kg/d'], 
                                                                                                                    diet_info.loc['Diet','Fd_DigrOMtIn'], diet_info.loc['Diet', 'Fd_CPIn'], diet_info.loc['Diet', 'Fd_RUPIn'],
                                                                                                                    diet_info.loc['Diet', 'Fd_idRUPIn'], diet_info.loc['Diet', 'Fd_NPNCPIn'], diet_info.loc['Diet', 'Fd_DigFAIn'], 
                                                                                                                    Du_MiCP_g, animal_input['An_BW'], animal_input['DMI'], nd.coeff_dict)

    # Predicted milk protein
    Mlk_NP_g, An_DigNDF, An_MPIn, An_DEInp = nd.calculate_Mlk_NP_g(AA_values, diet_info.loc['Diet', 'Fd_idRUPIn'], Du_idMiCP_g, An_DEIn, An_DETPIn, An_DENPNCPIn, An_DigNDFIn, An_DEStIn, An_DEFAIn, An_DErOMIn, An_DENDFIn, animal_input['An_BW'], animal_input['DMI'], animal_input['An_StatePhys'], nd.coeff_dict)

    # Net energy/Metabolizable energy
    An_NE, An_MEIn, Frm_NPgain = nd.calculate_An_NE(diet_info.loc['Diet', 'Fd_CPIn'], diet_info.loc['Diet', 'Fd_FAIn'], Mlk_NP_g, An_DEIn, An_DigNDF, Fe_CP, Fe_CPend_g, animal_input['DMI'], animal_input['An_BW'], animal_input['An_BW_mature'], 
                                     animal_input['Trg_FrmGain'], animal_input['Trg_RsrvGain'], GrUter_BWgain, nd.coeff_dict)

########################################
# Step 6: Requirement Calculations
########################################
    # Metabolizable Energy Requirements
    Trg_MEuse, An_MEmUse, An_MEgain, Gest_MEuse, Trg_Mlk_MEout, Trg_NEmilk_Milk = nd.calculate_ME_requirement(animal_input['An_BW'], animal_input['DMI'], animal_input['Trg_MilkProd'], animal_input['An_BW_mature'], animal_input['Trg_FrmGain'],
                                                                                          animal_input['Trg_MilkFatp'], animal_input['Trg_MilkTPp'], animal_input['Trg_MilkLacp'], animal_input['Trg_RsrvGain'], GrUter_BWgain, nd.coeff_dict)
    # Metabolizable Protein Requirements
    An_MPuse_g_Trg, An_MPm_g_Trg, Body_MPUse_g_Trg, Gest_MPUse_g_Trg, Mlk_MPUse_g_Trg = nd.calculate_MP_requirement(An_DEInp, An_DENPNCPIn, An_DigTPaIn, An_GasEOut, Frm_NPgain, diet_info.loc['Diet', 'Fd_NDFIn'], animal_input['DMI'], animal_input['An_BW'], animal_input['An_BW_mature'], animal_input['Trg_FrmGain'],
                                                                                                                 animal_input['Trg_RsrvGain'], animal_input['Trg_MilkProd'], animal_input['Trg_MilkTPp'], GrUter_BWgain, animal_input['An_StatePhys'], nd.coeff_dict)    

########################################
# Step 7: Performance Calculations
########################################

    # Predicted milk fat
    Mlk_Fat_g, An_LactDay_MlkPred = nd.calculate_Mlk_Fat_g(AA_values, diet_info.loc['Diet', 'Fd_FAIn'], diet_info.loc['Diet', 'Fd_DigC160In'], diet_info.loc['Diet', 'Fd_DigC183In'], animal_input['An_LactDay'], animal_input['DMI'])

    # Predicted milk yield
    Mlk_Prod_comp = nd.calculate_Mlk_Prod_comp(Mlk_NP_g, Mlk_Fat_g, An_DEIn, An_LactDay_MlkPred, animal_input['An_Parity_rl']) 
  
    # MP Allowable Milk
    Mlk_Prod_MPalow = nd.calculate_Mlk_Prod_MPalow(An_MPuse_g_Trg, Mlk_MPUse_g_Trg, An_MPIn, animal_input['Trg_MilkTPp'], nd.coeff_dict)

    # NE Allowable Milk
    Mlk_Prod_NEalow = nd.calculate_Mlk_Prod_NEalow(An_MEIn, An_MEgain, An_MEmUse, Gest_MEuse, Trg_NEmilk_Milk, nd.coeff_dict)

########################################
# Step 8: *Temporary* Add values of interest to list
########################################
    # Milk Fat %
    milk_fat = (Mlk_Fat_g / 1000) / Mlk_Prod_comp * 100
    # Milk Protein %
    milk_protein = (Mlk_NP_g / 1000) / Mlk_Prod_comp * 100

    model_results = {
    'milk_fat': milk_fat,
    'milk_protein': milk_protein,
    'Mlk_Prod_comp': Mlk_Prod_comp,
    'Mlk_Prod_MPalow': Mlk_Prod_MPalow,
    'Mlk_Prod_NEalow': Mlk_Prod_NEalow,
    'Trg_MEuse': Trg_MEuse,
    'An_MPuse_g_Trg': An_MPuse_g_Trg
}

    ####################
    # Testing Statements
    ####################
    

    return diet_info, animal_input, feed_data, equation_selection, AA_values, model_results

### RUN MODEL ###
diet_info, animal_input, feed_data, equation_selection, AA_values, model_results = run_dev_model()


In [ ]:
# New read input for input.csv
import pandas as pd

# input_test = pd.read_csv('../dev_scripts/input.csv')
# path = '../dev_scripts/input.csv'

def read_csv_input(path_to_file):
    
    animal_input = {}
    equation_selection = {}
    diet_info_data = {'Feedstuff': [], '%_DM_user': []}

    input_data = pd.read_csv(path_to_file)

    for index, row in input_data.iterrows():
        location = row['Location']
        variable = row['Variable']
        value = row['Value']
        
        if location == 'equation_selection':
            equation_selection[variable] = float(value) if value.replace('.', '', 1).isdigit() else value
        elif location == 'animal_input':
            animal_input[variable] = float(value) if value.replace('.', '', 1).isdigit() else value
        elif location == 'diet_info':
            diet_info_data['Feedstuff'].append(variable)
            diet_info_data['%_DM_user'].append(value)
      
    diet_info = pd.DataFrame(diet_info_data)
    diet_info['%_DM_user'] = pd.to_numeric(diet_info['%_DM_user'], downcast="float")

    return diet_info, animal_input, equation_selection

# Call function
# diet_info, animal_input, equation_selection = read_csv_input(path)
diet_info, animal_input, equation_selection = read_csv_input('./input.csv')

In [ ]:
# Heifer calculations, Line 317-379
# These values are only relavent for heifers so An_PhysState == "Heifer" should be checked before running

import math

# NEED TO TEST ALL VALUES
# Need to check DMI equation selection, most likely don't need to calculate everything, just the equation selected

def heifer_growth(Dt_NDF, An_BW, An_BW_mature, An_PrePartWk, coeff_dict):
# Lines 317-379
    req_coeffs = ['Ka_LateGest_DMIn', 'Kc_LateGest_DMIn']     
    check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)

#NRC 2020 Heifer Eqns. from the Transition Ch.
    Dt_NDFdev_DMI = Dt_NDF - (23.11 + 0.07968 * An_BW - 0.00006252 * An_BW**2)      # Line 316
#Animal factors only, eqn. 2-3 NRC
    Dt_DMIn_Heif_NRCa = 0.022 * An_BW_mature * (1 - math.exp(-1.54 * An_BW / AN_BW_mature))
#Holstein, animal factors only
    Dt_DMIn_Heif_H1 = 15.36 * (1 - math.exp(-0.0022 * An_BW))
#Holstein x Jersey, animal factors only
    Dt_DMIn_Heif_HJ1 = 12.91 * (1 - math.exp(-0.00295 * An_BW))
#Anim & diet factors, eqn 2-4 NRC
    Dt_DMIn_Heif_NRCad = (0.0226 * An_BW_mature * (1 - math.exp(-1.47 * An_BW / An_BW_mature))) - (0.082 * (Dt_NDF - (23.1 + 56 * An_BW / An_BW_mature) - 30.6 *(An_BW / An_BW_mature)**2))
#Holstein, animal factors and NDF
    Dt_DMIn_Heif_H2 = 15.79 * (1 - math.exp(-0.0021 * An_BW)) - (0.082 * Dt_NDFdev_DMI)
#Holstein x Jersey, animal factors and NDF
    Dt_DMIn_Heif_HJ2 = 13.48 * (1 - math.exp(-0.0027 * An_BW)) - (0.082 * Dt_NDFdev_DMI)

#Late Gestation eqn. from Hayirli et al., 2003) for dry cows and heifers
    if An_PrePartWk < -3:       #constrain to the interval 0 to -3.
        An_PrePartWklim = -3
    else:
        An_PrePartWklim = An_PrePartWk

    if An_PrePartWk > 0:
        An_PrePartWklim = 0

#the estimated length of time in the close-up pen
    An_PrePartWkDurat = An_PrePartWklim * 2  

    Dt_NDF_drylim = Dt_NDF

    if Dt_NDF < 30:    #constrain Dt_NDF to the range of 30 to 55% of DM
        Dt_NDF_drylim = 30
    if Dt_NDF > 55:
        Dt_NDF_drylim = 55

    Kb_LateGest_DMIn = -(0.365 - 0.0028 * Dt_NDF_drylim)

#Late gestation individual animal prediction, % of BW.  Use to assess for a specific day for a given animal
    Dt_DMIn_BW_LateGest_i = coeff_dict['Ka_LateGest_DMIn'] + Kb_LateGest_DMIn * An_PrePartWklim + coeff_dict['Kc_LateGest_DMIn'] * An_PrePartWklim**2
#Late gestation Group/Pen mean DMI/BW for an interval of 0 to PrePart_WkDurat.  Assumes pen steady state and PrePart_wk = pen mean
    Dt_DMIn_BW_LateGest_p = (coeff_dict['Ka_LateGest_DMIn'] * An_PrePartWkDurat + Kb_LateGest_DMIn / 2 * An_PrePartWkDurat**2 + coeff_dict['Kc_LateGest_DMIn'] / 3 * An_PrePartWkDurat**3) / An_PrePartWkDurat
#Individual intake for the specified day prepart or the pen mean intake for the interval, 0 to PrePart_WkDurat
    Dt_DMIn_Heif_LateGestInd = 0.88 * An_BW * Dt_DMIn_BW_LateGest_i / 100 #Individual animal
    Dt_DMIn_Heif_LateGestPen = 0.88 * An_BW * Dt_DMIn_BW_LateGest_p / 100 #Pen mean

#Switch to the group transition eqn. when less than An_PrePartWkDurat and predicted transition DMI is less than far off DMI
#These equations generally are discontinuous at -3 weeks, as Dt_DMIn_xxx is greater than Dt_DMIn_xxx_LateGest at -3.  
#Should calculate the decline using the far off DMIn as a reference point, or calculate the discontinuity at the
#point of transition and adjust the close-up DMIn by addition of the gap.

#NRC 2020 pen intakes
    if An_PrePartWk > An_PrePartWkDurat:
        Dt_DMIn_Heif_NRCap = min(Dt_DMIn_Heif_NRCa, Dt_DMIn_Heif_LateGestPen)
        Dt_DMIn_Heif_NRCadp = min(Dt_DMIn_Heif_NRCad, Dt_DMIn_Heif_LateGestPen)
        Dt_DMIn_Heif_H1p =  min(Dt_DMIn_Heif_H1, Dt_DMIn_Heif_LateGestPen)
        Dt_DMIn_Heif_HJ1p = min(Dt_DMIn_Heif_HJ1, Dt_DMIn_Heif_LateGestPen)
        Dt_DMIn_Heif_H2p = min(Dt_DMIn_Heif_H2, Dt_DMIn_Heif_LateGestPen)
        Dt_DMIn_Heif_HJ2p = min(Dt_DMIn_Heif_HJ2, Dt_DMIn_Heif_LateGestPen)
    else:
        Dt_DMIn_Heif_NRCap = Dt_DMIn_Heif_NRCa
        Dt_DMIn_Heif_NRCadp = Dt_DMIn_Heif_NRCad
        Dt_DMIn_Heif_H1p = Dt_DMIn_Heif_H1
        Dt_DMIn_Heif_HJ1p = Dt_DMIn_Heif_HJ1
        Dt_DMIn_Heif_H2p = Dt_DMIn_Heif_H2
        Dt_DMIn_Heif_HJ2p = Dt_DMIn_Heif_HJ2

#NRC 2020 individual animal intakes
    if An_PrePartWk > An_PrePartWkDurat:
        Dt_DMIn_Heif_NRCai = min(Dt_DMIn_Heif_NRCa, Dt_DMIn_Heif_LateGestInd)
        Dt_DMIn_Heif_NRCadi =  min(Dt_DMIn_Heif_NRCad, Dt_DMIn_Heif_LateGestInd)
        Dt_DMIn_Heif_H1i = min(Dt_DMIn_Heif_H1, Dt_DMIn_Heif_LateGestInd)
        Dt_DMIn_Heif_HJ1i = min(Dt_DMIn_Heif_HJ1, Dt_DMIn_Heif_LateGestInd)
        Dt_DMIn_Heif_H2i = min(Dt_DMIn_Heif_H2, Dt_DMIn_Heif_LateGestInd)
        Dt_DMIn_Heif_HJ2i = min(Dt_DMIn_Heif_HJ2,Dt_DMIn_Heif_LateGestInd)
    else:
        Dt_DMIn_Heif_NRCai = Dt_DMIn_Heif_NRCa
        Dt_DMIn_Heif_NRCadi = Dt_DMIn_Heif_NRCad
        Dt_DMIn_Heif_H1i = Dt_DMIn_Heif_H1
        Dt_DMIn_Heif_HJ1i = Dt_DMIn_Heif_HJ1
        Dt_DMIn_Heif_H2i = Dt_DMIn_Heif_H2
        Dt_DMIn_Heif_HJ2i = Dt_DMIn_Heif_HJ2
